In [1]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-08-02 18:53:39,997 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-02 18:53:40,024 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-02 18:53:40,069 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-02 18:53:40,071 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-02 18:53:40,077 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-02 18:53:40,078 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-02 18:53:40,081 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-02 18:53:40,082 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-02 18:53:4

In [2]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "pending_enrichment" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [3]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-02 18:53:40,247 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-02 18:53:40,290 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [4]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[transform name here]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [116]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

    sort_columns: list # List of Long-ID, Pharmacy Code, Brand/Medication and Status Date used to sort the dataframe. Order of variables matters
    substatus_list: list # List of integrichain substatuses for current customer. Order of variables does not matter.
    pjh: str # Patient Journey Hiearchy column, should have been added in an early transform
    ic_status: str # Integrichain Status column, should have been added in an early transform
    ic_substatus: str # Integrichain Sub Status column, should have been added in an early transform
    bvpa: str # BV/PA string in Patient Journey Hierarchy column. String should be something like 'BV/PA'
    intake: str # Intake string in Patient Journey Hierarchy column. String should be something like 'INTAKE'
    fulfillment: str # Fulfillment string in Patient Journey Hierarchy column. String should be something like 'FULFILLMENT'
    pending: str # Pending string in Integrichain Status column. String should be something like 'PENDING'
    
    
    def pending_enrichment(self,df):

        df = df.sort_values(self.sort_columns,ascending=[True, True, True, True])

        groupby_cols = self.sort_columns[:-1]
        status_date = self.sort_columns[-1]
        
        min_bvpa, max_bvpa, max_bvpa_intake, min_bvpa_fulfillment = self._create_bvpa(df=df, groupby_cols=groupby_cols, status_date=status_date)
        
        df = pd.merge(df, min_bvpa_fulfillment, how='left', on=groupby_cols)
        df = pd.merge(df, min_bvpa, how='left', on=groupby_cols)
        df = pd.merge(df, max_bvpa, how='left', on=groupby_cols)
        df = pd.merge(df, max_bvpa_intake, how='left', on=groupby_cols)

        df.loc[:,'Before_min_fulfillment'] = np.where(df.loc[:,status_date] < df.loc[:,'min_bvpa_fulfillment'],1,0)
        df.loc[:,'After_min_BVPA'] = np.where(df.loc[:,status_date] >= df.loc[:,'min_bvpa_date'],1,0)
        df.loc[:,'Before_max_BVPA'] = np.where(df.loc[:,status_date] <= df.loc[:,'max_bvpa_date'],1,0)
        df.loc[:,'After_max_intake'] = np.where(df.loc[:,status_date] > df.loc[:,'max_bvpa_intake_date'],1,0)
        
        df.loc[(df.Before_min_fulfillment == 0) & (df.After_min_BVPA == 0) & (df.Before_max_BVPA == 1), 'Before_min_fulfillment'] = 1
        df.loc[(df.Before_min_fulfillment == 1) & (df.After_min_BVPA == 0) & (df.Before_max_BVPA == 1), 'Before_max_BVPA'] = 0

        df.loc[(df.After_max_intake == 0) & (df.Before_max_BVPA == 0) & (df.After_min_BVPA == 1), 'After_max_intake'] = 1
        df.loc[(df.After_max_intake == 1) & (df.Before_max_BVPA == 0) & (df.After_min_BVPA == 1), 'After_min_BVPA'] = 0
        
        # 820 Due to this split and 
        enrich_df = df[(df[self.ic_status] == self.pending) & (df[self.ic_substatus].isin(self.substatus_list))]
        df = df[~((df[self.ic_status] == self.pending) & (df[self.ic_substatus].isin(self.substatus_list)))]
        
        # 820 Due to this fill method
        enrich_df.loc[:,self.pjh] = (
            np.where(enrich_df.loc[:,'Before_min_fulfillment'] == 1, self.intake,
            np.where((enrich_df.loc[:,'After_min_BVPA'] == 1) & (enrich_df.loc[:,'Before_max_BVPA'] == 1),self.bvpa,
            np.where(enrich_df.loc[:,'After_max_intake'] == 1, self.fulfillment,
            enrich_df.loc[:,self.pjh])))
        )
        
        df = pd.concat([df,enrich_df],sort=False)
        
        final_df = df.drop(labels=['After_min_BVPA','After_max_intake','Before_max_BVPA','Before_min_fulfillment','min_bvpa_date','max_bvpa_intake_date','max_bvpa_date','min_bvpa_fulfillment'],axis=1)
        
        return final_df,df
    
        
    def _create_bvpa(self, df, groupby_cols, status_date):
        
        bvpa_df = df[(df[self.pjh] == self.bvpa)]

        intake_df = df[(df[self.pjh].isin([self.bvpa,self.intake]))]

        fulfillment_df = df[(df[self.pjh].isin([self.bvpa,self.fulfillment]))]

        min_bvpa = (
            bvpa_df
            .groupby(groupby_cols)[status_date]
            .min()
            .reset_index(drop=False)
            .rename(columns={status_date:'min_bvpa_date'})
        )
        
        max_bvpa = (
            bvpa_df
            .groupby(groupby_cols)[status_date]
            .max()
            .reset_index(drop=False)
            .rename(columns={status_date:'max_bvpa_date'})
        )
        
        max_bvpa_intake = (
            intake_df
            .groupby(groupby_cols)[status_date]
            .max()
            .reset_index(drop=False)
            .rename(columns={status_date:'max_bvpa_intake_date'})
        )
        
        min_bvpa_fulfillment = (
            fulfillment_df
            .groupby(groupby_cols)[status_date]
            .min()
            .reset_index(drop=False)
            .rename(columns={status_date:'min_bvpa_fulfillment'})
        )

        return min_bvpa, max_bvpa, max_bvpa_intake, min_bvpa_fulfillment
    
transform = Transform()

In [117]:
status_config[status_config.integrichain_status == 'PENDING']

,status_code,sub_status,integrichain_sub_status,integrichain_status,Patient_Journey_Hierarchy
0,PENDING,NEW,NEW,PENDING,INTAKE
1,PENDING,BENEFITS,BENEFITS,PENDING,BV/PA
2,PENDING,PA,PA,PENDING,BV/PA
3,PENDING,PATIENT HOLD,PATIENT HOLD,PENDING,FULFILLMENT
4,PENDING,PATIENT FINANCIAL,PATIENT FINANCIAL,PENDING,FINANCIAL
5,PENDING,PRESCRIBER HOLD,PRESCRIBER HOLD,PENDING,FULFILLMENT
6,PENDING,PATIENT CONTACT,PATIENT CONTACT,PENDING,FULFILLMENT
7,PENDING,INVENTORY HOLD,INVENTORY HOLD,PENDING,FULFILLMENT
8,PENDING,FOUNDATION,FOUNDATION,PENDING,FINANCIAL
9,PENDING,THERAPY HOLD,THERAPY HOLD,PENDING,FULFILLMENT


In [118]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

transform.sort_columns = ['msa_patient_id', 'pharm_code', 'medication', 'status_date']
transform.pjh = 'Patient_Journey_Hierarchy'
transform.ic_status = 'integrichain_status'
transform.ic_substatus = 'integrichain_sub_status'
transform.bvpa = 'BV/PA'
transform.intake = 'INTAKE'
transform.fulfillment = 'FULFILLMENT'
transform.pending = 'PENDING'
transform.substatus_list = ['OTHER','PATIENT CONTACT','PATIENT RESPONSE','PATIENT HOLD','PATIENT FINANCIAL','PRESCRIBER','PRESCRIBER HOLD','READY']

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

![Pending Enrichment Flowchart](assets/PendingEnrichment.png)

Replaces ambiguous substatus with info based on sub-statuses surrounding the ambiguous field

### Transformation

In [119]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
# run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[1]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
df = input_contract.fetch(filters=run_filter)

import numpy as np
import pandas as pd
pd.options.display.max_columns=999
# pd.options.display.max_rows=999

df.status_date = df.status_date.str[:8].astype(str)
df.ref_date = df.ref_date.str[:8].astype(str)

df.status_date = pd.to_datetime(df.status_date, infer_datetime_format=True, errors='coerce')
df.ref_date = pd.to_datetime(df.ref_date, infer_datetime_format=True, errors='coerce')

2019-08-02 20:30:15,813 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


In [120]:
import os
os.chdir('{}'.format(os.path.expanduser('~')))
status_config = pd.read_csv('status_mapping.csv')

status_config.loc[:,'statusCode'] = status_config.statusCode.str.upper()
status_config.loc[:,'subStatus'] = status_config.subStatus.str.upper()
status_config.loc[:,'integrichain_sub_status'] = status_config.integrichain_sub_status.str.upper()
status_config.loc[:,'integrichain_status'] = status_config.integrichain_status.str.upper()
status_config.loc[:,'Patient_Journey_Hierarchy'] = status_config.Patient_Journey_Hierarchy.str.upper()

status_config = status_config.rename(columns={'statusCode':'status_code','subStatus':'sub_status'})

df.sub_status = df.sub_status.str.replace('PRESCRIBERHOLD','PRESCRIBER HOLD')

df = pd.merge(df,status_config,on=['status_code','sub_status'])

df = df[['rec_date', 'pharm_code', 'pharm_npi', 'transtype', 'pharm_transaction_id', 'trans_seq', 'ref_source', 'ref_date', 'program_id', 'pharmacy_id', 'pat_last_name', 'pat_first_name', 'pat_dob', 'pat_gender', 
         'pat_addr1', 'pat_addr2', 'pat_city', 'pat_state', 'pat_zip', 'dx1_code', 'dx2_code', 'status_date', 'status_code', 'sub_status', 'integrichain_status','integrichain_sub_status', 'Patient_Journey_Hierarchy', 
         'pres_last_name', 'pres_first_name', 'pres_addr1', 'pres_addr2', 'pres_city', 'pres_state', 'pres_zip', 'pres_phone', 'pres_npi', 'pres_dea', 'facility_name', 'rxdate', 'rxnumber', 'rxrefills', 'rxfill', 
         'refill_remaining', 'prev_disp', 'rx_ndc_number', 'medication', 'quantity', 'day_supply', 'ship_date', 'ship_carrier', 'shiptracking_num', 'ship_location', 'ship_address', 'ship_city', 'ship_state', 'ship_zip', 
         'has_medical', 'primary_coverage_type', 'primary_payer_name', 'primary_payer_type', 'secondary_coverage_type', 'secondary_payer_name', 'secondary_payer_type', 'plan_paid_amt', 'pat_copay', 'copay_assist_amount', 
         'oth_payer_amt', 'xfer_pharmname', 'msa_patient_id', 'msa_patient_bmap', '__metadata_run_timestamp', '__metadata_app_version', '__metadata_output_contract', '__metadata_transform_timestamp', '__metadata_run_id']]

In [121]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe
final_dataframe, unittest_df = transform.pending_enrichment(df)

In [122]:
import unittest

def shape_status(final_dataframe,df):
    return df.shape[0] == final_dataframe.shape[0]

def dummy_fail(df):
    df['binary_series_1'] = df.Before_min_fulfillment + df.Before_max_BVPA + df.After_max_intake
    df['binary_series_2'] = df.Before_min_fulfillment + df.After_min_BVPA + df.After_max_intake
    return df[(df.binary_series_1 > 1) & (df.binary_series_2 > 1)].shape[0] == 0

def between_status(df):
    df['binary_series'] = df.Before_max_BVPA + df.After_min_BVPA
    return df[(unittest_df.binary_series == 1)].shape[0] == 0

class TestNotebook(unittest.TestCase):
    
    def test_shape_status(self):
        self.assertEqual(shape_status(final_dataframe,df),True)
        
    def test_dummy_fail(self):
        self.assertEqual(dummy_fail(unittest_df),True)
        
    def test_between_status(self):
        self.assertEqual(between_status(unittest_df),True)
    
unittest.main(argv=[''],verbosity=2,exit=False)

test_between_status (__main__.TestNotebook) ... ok
test_dummy_fail (__main__.TestNotebook) ... ok
test_shape_status (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.030s

OK


In [123]:
df.head()

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,pat_last_name,pat_first_name,pat_dob,pat_gender,pat_addr1,pat_addr2,pat_city,pat_state,pat_zip,dx1_code,dx2_code,status_date,status_code,sub_status,integrichain_status,integrichain_sub_status,Patient_Journey_Hierarchy,pres_last_name,pres_first_name,pres_addr1,pres_addr2,pres_city,pres_state,pres_zip,pres_phone,pres_npi,pres_dea,facility_name,rxdate,rxnumber,rxrefills,rxfill,refill_remaining,prev_disp,rx_ndc_number,medication,quantity,day_supply,ship_date,ship_carrier,shiptracking_num,ship_location,ship_address,ship_city,ship_state,ship_zip,has_medical,primary_coverage_type,primary_payer_name,primary_payer_type,secondary_coverage_type,secondary_payer_name,secondary_payer_type,plan_paid_amt,pat_copay,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id
0,20181024115959,ACCREDO,1346208949,COM,279133432018102401,0,DIRECT,2018-10-19,None,27913343,None,None,None,None,None,None,None,None,00,L40.0,None,2018-10-24,CANCELLED,OTHER,CANCELLED,OTHER,PROVIDER,None,None,None,None,None,None,99999,None,None,None,None,None,None,None,None,None,None,None,ILUMYA,None,None,None,None,None,None,None,None,None,None,Y,MEDICAL,GENERAL DIRECT,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNNV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
1,20181029115959,ACCREDO,1346208949,COM,279181482018102903,0,DIRECT,2018-10-24,None,27918148,None,None,None,M,None,None,None,None,00,L40.0,None,2018-10-29,CANCELLED,OTHER,CANCELLED,OTHER,PROVIDER,SCIURBA,SALVATORE,111 WEST WATER ST,None,TOMS RIVER,NJ,08753,7322444700,1093765307,None,None,None,None,None,None,None,None,None,ILUMYA,None,None,None,None,None,None,None,None,None,None,Y,MEDICAL,GENERAL HORIZON BCBS NJ,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
2,20181106115959,ACCREDO,1346208949,COM,160618142018110605,0,DIRECT,2018-11-02,None,16061814,None,None,None,F,None,None,None,None,00,696.1,None,2018-11-06,CANCELLED,OTHER,CANCELLED,OTHER,PROVIDER,KORY,MARK,16216 BAXTER ROAD,SUITE 200,CHESTERFIELD,MO,63017,6365321000,1326034489,BK1220045,None,None,None,None,None,None,None,None,ILUMYA,None,None,None,None,None,None,None,None,None,None,Y,MEDICAL,EXPRESS SCRIPTS,COMMERCIAL,None,None,None,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
3,20181108115959,ACCREDO,1346208949,COM,160850092018110807,0,DIRECT,2018-11-08,None,16085009,None,None,None,M,None,None,None,None,00,L40.0,None,2018-11-08,CANCELLED,OTHER,CANCELLED,OTHER,PROVIDER,SONG,EIN,15906 MILL CREEK BLVD,None,BOTHELL,WA,98012,4253852009,1548536972,FS4902777,None,None,None,None,None,None,None,None,ILUMYA,None,None,None,None,None,None,None,None,None,None,Y,PHARMACY,MEDCO HOME DELIVERY,COMMERCIAL,MEDICAL,PSKW COPAY PROGRAM-COSENTYX-WEBMD,CASH,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
4,20181109115959,ACCREDO,1346208949,COM,270756892018110908,0,DIRECT,2018-11-08,None,27075689,None,None,None,F,None,None,None,None,00,None,None,2018-11-09,CANCELLED,OTHER,CANCELLED,OTHER,PROVIDER,KUMAR,ASHWIN,252 BROAD ST,None,CLAREMONT,NH,03743,6035426455,1275931776,MK3414062,None,None,None,None,None,None,None,None,ILUMYA,None,None,None,None,None,None,None,None,None,None,Y,PHARMACY,MEDCO HOME DELIVERY,COMMERCIAL,MEDICAL,MCKESSON LOYALTY,CASH,None,None,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3


unique_id_dict = (
    final_dataframe[(final_dataframe.integrichain_status == 'PENDING') 
                    & (final_dataframe.integrichain_sub_status.isin(transform.substatus_list))]
    [['msa_patient_id','pharm_code','medication']]
    .drop_duplicates()
    .reset_index(drop=True)
    .to_dict(orient='index')
)

In [124]:
final_dataframe = final_dataframe.sort_values(transform.sort_columns,ascending=[True, True, True, True])
unittest_df = unittest_df.sort_values(transform.sort_columns,ascending=[True, True, True, True])
df = df.sort_values(transform.sort_columns,ascending=[True, True, True, True])

In [125]:
from random import randint

In [126]:
i
#774

820

# 820
# {'msa_patient_id': '4700022',
#  'pharm_code': 'WAG',
#  'medication': 'ILUMYA 100MG/ML PFS 1ML'}

# 845
# {'msa_patient_id': '4830004',
#  'pharm_code': 'WAG',
#  'medication': 'ILUMYA 100MG/ML PFS 1ML'}

In [127]:
unique_id_dict[i]

{'msa_patient_id': '4700022',
 'pharm_code': 'WAG',
 'medication': 'ILUMYA 100MG/ML PFS 1ML'}

In [128]:
transform.substatus_list

['OTHER',
 'PATIENT CONTACT',
 'PATIENT RESPONSE',
 'PATIENT HOLD',
 'PATIENT FINANCIAL',
 'PRESCRIBER',
 'READY']

In [129]:
i = 820
unittest_df[(unittest_df.msa_patient_id == unique_id_dict[i]['msa_patient_id'])
            & (unittest_df.pharm_code == unique_id_dict[i]['pharm_code'])
            & (unittest_df.medication == unique_id_dict[i]['medication'])]

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,pat_last_name,pat_first_name,pat_dob,pat_gender,pat_addr1,pat_addr2,pat_city,pat_state,pat_zip,dx1_code,dx2_code,status_date,status_code,sub_status,integrichain_status,integrichain_sub_status,Patient_Journey_Hierarchy,pres_last_name,pres_first_name,pres_addr1,pres_addr2,pres_city,pres_state,pres_zip,pres_phone,pres_npi,pres_dea,facility_name,rxdate,rxnumber,rxrefills,rxfill,refill_remaining,prev_disp,rx_ndc_number,medication,quantity,day_supply,ship_date,ship_carrier,shiptracking_num,ship_location,ship_address,ship_city,ship_state,ship_zip,has_medical,primary_coverage_type,primary_payer_name,primary_payer_type,secondary_coverage_type,secondary_payer_name,secondary_payer_type,plan_paid_amt,pat_copay,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id,min_bvpa_fulfillment,min_bvpa_date,max_bvpa_date,max_bvpa_intake_date,Before_min_fulfillment,After_min_BVPA,Before_max_BVPA,After_max_intake,binary_series,binary_series_1,binary_series_2
29998,20190517 03:21:30,WAG,1457618555,COM,563535185952,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,None,None,2019-05-16,PENDING,PRESCRIBER,PENDING,PRESCRIBER,INTAKE,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,None,None,OTHER,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3,2019-05-16,2019-05-19,2019-06-21,2019-06-21,1,0,0,0,0,1,1
29999,20190517 03:21:30,WAG,1457618555,COM,563535185952,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,None,None,2019-05-16,PENDING,PRESCRIBER,PENDING,PRESCRIBER,INTAKE,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,None,None,OTHER,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3,2019-05-16,2019-05-19,2019-06-21,2019-06-21,1,0,0,0,0,1,1
30000,20190517 03:21:30,WAG,1457618555,COM,563535185952,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,None,None,2019-05-16,PENDING,PRESCRIBER,PENDING,PRESCRIBER,INTAKE,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,None,None,OTHER,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:31,4,2019-05-16,2019-05-19,2019-06-21,2019-06-21,1,0,0,0,0,1,1
30001,20190520 01:40:09,WAG,1457618555,COM,553235489496,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,L40.0,None,2019-05-19,PENDING,BENEFITS,PENDING,BENEFITS,BV/PA,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,P,OPTUM RX,COMMERCIAL,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3,2019-05-16,2019-05-19,2019-06-21,2019-06-21,0,1,1,0,2,1,1
30002,20190520 01:40:09,WAG,1457618555,COM,553235489496,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,L40.0,None,2019-05-19,PENDING,BENEFITS,PENDING,BENEFITS,BV/PA,KOONE,MARK,815 PECAN GROVE RD E,None,

In [134]:
status_config.integrichain_sub_status.str.isin('TRANSFER')

AttributeError: 'StringMethods' object has no attribute 'isin'

In [76]:
df[(df.msa_patient_id == unique_id_dict[i]['msa_patient_id'])
   & (df.pharm_code == unique_id_dict[i]['pharm_code'])
   & (df.medication == unique_id_dict[i]['medication'])]

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,pat_last_name,pat_first_name,pat_dob,pat_gender,pat_addr1,pat_addr2,pat_city,pat_state,pat_zip,dx1_code,dx2_code,status_date,status_code,sub_status,integrichain_status,integrichain_sub_status,Patient_Journey_Hierarchy,pres_last_name,pres_first_name,pres_addr1,pres_addr2,pres_city,pres_state,pres_zip,pres_phone,pres_npi,pres_dea,facility_name,rxdate,rxnumber,rxrefills,rxfill,refill_remaining,prev_disp,rx_ndc_number,medication,quantity,day_supply,ship_date,ship_carrier,shiptracking_num,ship_location,ship_address,ship_city,ship_state,ship_zip,has_medical,primary_coverage_type,primary_payer_name,primary_payer_type,secondary_coverage_type,secondary_payer_name,secondary_payer_type,plan_paid_amt,pat_copay,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id
26551,20190517 03:21:30,WAG,1457618555,COM,563535185952,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,None,None,2019-05-16,PENDING,PRESCRIBER,PENDING,PRESCRIBER,FULFILLMENT,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,None,None,OTHER,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3
26609,20190517 03:21:30,WAG,1457618555,COM,563535185952,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,None,None,2019-05-16,PENDING,PRESCRIBER,PENDING,PRESCRIBER,FULFILLMENT,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,None,None,OTHER,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3
26946,20190517 03:21:30,WAG,1457618555,COM,563535185952,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,None,None,2019-05-16,PENDING,PRESCRIBER,PENDING,PRESCRIBER,FULFILLMENT,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,None,None,OTHER,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-06-26 15:28:20,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-06-26 15:33:31,4
31914,20190520 01:40:09,WAG,1457618555,COM,553235489496,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,L40.0,None,2019-05-19,PENDING,BENEFITS,PENDING,BENEFITS,BV/PA,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,P,OPTUM RX,COMMERCIAL,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3
32820,20190520 01:40:09,WAG,1457618555,COM,553235489496,0,DIRECT,2019-05-15,None,355848517441,None,None,None,F,None,None,None,None,None,L40.0,None,2019-05-19,PENDING,BENEFITS,PENDING,BENEFITS,BV/PA,KOONE,MARK,815 PECAN GROVE RD E,None,SHERMAN,TX,75090,9038922126,1720055833,BK1733143,None,20190412,None,1,None,None,None,47335017795,ILUMYA 100MG/ML PFS 1ML,None,28,None,None,None,None,None,None,None,None,N,P,OPTUM RX,COMMERCIAL,None,None,None,None,None,None,None,None,4700022,VVVVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:27,3
34197,20190520 01:40:09,WAG,1457618555,COM,553235489496,0,

In [18]:
# Fulfill <= Intake >=
final_dataframe.Patient_Journey_Hierarchy.value_counts()
# Fails test_dummy_fail

FULFILLMENT    14072
BV/PA          12247
INTAKE          3812
PAYER           2748
PROVIDER        2541
PATIENT         1460
TRANSFERRED      381
FINANCIAL        144
Name: Patient_Journey_Hierarchy, dtype: int64

In [39]:
# BVPA, Fulfillment <=
final_dataframe.Patient_Journey_Hierarchy.value_counts()
# Fails test_dummy_fail

FULFILLMENT    14069
BV/PA          12250
INTAKE          3812
PAYER           2748
PROVIDER        2541
PATIENT         1460
TRANSFERRED      381
FINANCIAL        144
Name: Patient_Journey_Hierarchy, dtype: int64

In [25]:
# BVPA <=, >=
final_dataframe.Patient_Journey_Hierarchy.value_counts()

FULFILLMENT    15334
BV/PA          12256
PAYER           2748
PROVIDER        2541
INTAKE          2541
PATIENT         1460
TRANSFERRED      381
FINANCIAL        144
Name: Patient_Journey_Hierarchy, dtype: int64

In [32]:
# BVPA, Intake >=
final_dataframe.Patient_Journey_Hierarchy.value_counts()

FULFILLMENT    15340
BV/PA          12250
PAYER           2748
PROVIDER        2541
INTAKE          2541
PATIENT         1460
TRANSFERRED      381
FINANCIAL        144
Name: Patient_Journey_Hierarchy, dtype: int64

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()